In [ ]:
import shutil
from requests import get

from google.colab import drive, files
# mount Google Drive
drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
!nvidia-smi

Wed Mar  8 22:10:07 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   68C    P0    32W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!pip -q install pymatgen

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.5/12.5 MB 67.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.9/66.9 KB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 515.1/515.1 KB 35.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 561.4/561.4 KB 22.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.5/109.5 KB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.4/71.4 KB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.4/98.4 KB 12.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.0/137.0 KB 17.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 555.3/555.3 KB 50.9 MB/s eta 0:00:00


In [ ]:
import re
import os
import pandas as pd
import numpy as np
import random
import joblib
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
import pickle
from sklearn.metrics import mean_squared_error 
from sklearn.model_selection import KFold

import warnings
warnings.simplefilter("ignore")

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torch.optim.lr_scheduler import StepLR
from pymatgen.core import Structure

ModuleNotFoundError: ignored

In [ ]:
class CFG:
  base_dir = "/content/drive/MyDrive/Colab Notebooks/Geo_CGCNN"
  data_dir = os.path.join(base_dir, "nishika_csv")
  output_dir_cif = os.path.join(base_dir, "database/cif")
  output_dir_target = os.path.join(base_dir, "database")
  sub_dir = os.path.join(base_dir, "data")
  target_col = "formation_energy_per_atom"

# テストデータの推論、提出データ作成

In [ ]:
sub_df = pd.read_csv(f"{CFG.sub_dir}/test_predictions_6_8_12.csv")
sub_df.head()

,name,prediction,target
0,mp-71730,-2.637851,0.5
1,mp-71731,-0.520380,0.5
2,mp-71732,-3.267011,0.5
3,mp-71733,-0.620214,0.5
4,mp-71734,-3.220563,0.5


In [ ]:
sub_df.drop("target", axis = 1,inplace=True)
sub_df.rename(columns={'name': 'id','prediction': 'formation_energy_per_atom'},inplace=True)
sub_df.head()

,id,formation_energy_per_atom
0,mp-71730,-2.637851
1,mp-71731,-0.520380
2,mp-71732,-3.267011
3,mp-71733,-0.620214
4,mp-71734,-3.220563


In [ ]:
for i in range(len(sub_df)):
  sub_df.loc[sub_df.index[i],"id"]=int(sub_df.loc[sub_df.index[i],"id"][3:])
sub_df.head()

,id,formation_energy_per_atom
0,71730,-2.637851
1,71731,-0.520380
2,71732,-3.267011
3,71733,-0.620214
4,71734,-3.220563


In [ ]:
a=df1 = pd.DataFrame(data={'id': [71730], 'formation_energy_per_atom': [-2.1]})

In [ ]:
sub_df=pd.concat([sub_df,a],ignore_index=True)

In [ ]:
sub_df.sort_values('id',inplace=True)
sub_df.head()

,id,formation_energy_per_atom
0,71730,-2.637851
1,71731,-0.520380
2,71732,-3.267011
3,71733,-0.620214
4,71734,-3.220563


In [ ]:
sub_df.to_csv(f"{CFG.sub_dir}/submission.csv", index = False)
pd.read_csv(f"{CFG.sub_dir}/submission.csv").head()

,id,formation_energy_per_atom
0,71730,-2.637851
1,71731,-0.520380
2,71732,-3.267011
3,71733,-0.620214
4,71734,-3.220563
